<a href="https://colab.research.google.com/github/biz-HK/inspection_ai_benchmark.ipynb/blob/main/inspection_ai_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔬 外観検査AI ベンチマーク
## Sparse Autoencoder vs Deep Autoencoder 比較検証

| 項目 | 内容 |
|------|------|
| **目的** | スパースモデルとディープラーニングモデルの異常検知性能を同一条件で比較 |
| **環境** | Google Colab (T4 GPU) / Windows (CPU/CUDA) / Mac (MPS) |
| **データ** | 合成データ（実データ差し替え可能） |
| **評価指標** | AUC-ROC, F1, FPR@95TPR, 推論速度, モデルサイズ |

---
### 実行手順
1. 「ランタイム」→「ランタイムのタイプを変更」→ **T4 GPU** を選択
2. 上から順にセルを実行（`Shift+Enter`）
3. 結果は自動的にGoogle Driveに保存されます

## 前提知識: このノートブックで何をしているのか

### 外観検査AIとは？
製造業の工場では、製品に**傷・汚れ・欠け・変色**などの不良（欠陥）がないかを検査する工程があります。
従来は人の目で行っていたこの作業を、AIに画像を見せて自動判定させるのが**外観検査AI**です。

### 異常検知のアプローチ: Autoencoder（オートエンコーダ）
このノートブックでは **Autoencoder** という手法を使っています。基本的な考え方は以下の通りです。

```
        Encoder           Decoder
入力画像 ──→ [圧縮表現(潜在変数)] ──→ 再構成画像
 (128×128)      (64 or 128次元)       (128×128)
```

1. **学習フェーズ**: 「正常品だけ」を大量に見せて、入力画像をそっくりそのまま出力できるように学習する
2. **推論フェーズ**: 未知の画像を入力したとき…
   - **正常品** → 学習済みパターンに近いので、うまく再構成できる（入力と出力の差が**小さい**）
   - **異常品** → 見たことのないパターンなので、うまく再構成できない（入力と出力の差が**大きい**）

この「再構成誤差（入力と出力の差）」を**異常スコア**として使い、閾値を超えたら「異常」と判定します。

### なぜ正常品だけで学習するのか？
製造現場では不良品のデータは極端に少ないのが普通です（不良率1%以下の場合も多い）。
Autoencoderなら**正常品だけで学習**でき、「正常から外れたもの＝異常」として検出できるため、
現場で非常に使いやすい手法です。

### このノートブックの目的
以下の**2種類のAutoencoder**を同じ条件で比較し、どちらが外観検査に適しているかを検証します。

| モデル | 特徴 | 向いている用途 |
|--------|------|---------------|
| **Sparse Autoencoder (SAE)** | 軽量・高速。少数の特徴に集中して学習 | エッジデバイス（カメラ付き検査機）での高速判定 |
| **Deep Autoencoder (DAE)** | 大規模・高精度。微細な異常も検出可能 | サーバーでの高精度バッチ検査 |

### セクション0の解説: 環境セットアップで行うこと

このセルでは、ノートブック全体で使うライブラリの読み込みと初期設定を行います。

**主要ライブラリの役割:**
- `torch` (PyTorch): ディープラーニングのフレームワーク。モデルの定義・学習・推論に使う
- `numpy`: 数値計算ライブラリ。画像データの配列操作に使う
- `sklearn` (scikit-learn): 機械学習の評価指標（AUC-ROCなど）を計算するために使う
- `matplotlib`: グラフや画像の描画ライブラリ

**デバイス自動検出について:**
PyTorchでは計算を行うハードウェア（デバイス）を指定できます。
- **CUDA (GPU)**: NVIDIA製GPU。行列演算を並列化でき、学習が10〜100倍高速になる
- **MPS**: Apple Silicon (M1/M2/M3/M4/M5) チップのGPU加速
- **CPU**: GPUがない場合のフォールバック。動作はするが遅い

**カスタムカラーマップ:**
異常箇所をヒートマップで可視化するための色の定義です（青=正常、黄=やや異常、赤=異常大）。

## 0. 環境セットアップ

In [ ]:
# ── 依存パッケージ（Colabにプリインストール済みのものが多い）──
!pip install -q scikit-learn matplotlib

import torch
import torch.nn as nn
import numpy as np
import time
import json
import os
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score, f1_score, roc_curve, precision_recall_curve
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.gridspec import GridSpec
from typing import Tuple, Dict, List
import warnings
warnings.filterwarnings('ignore')

# ── デバイス自動検出 ──
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    print(f'✅ GPU: {torch.cuda.get_device_name(0)}')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    DEVICE = torch.device('mps')
    print('✅ Apple Silicon MPS')
else:
    DEVICE = torch.device('cpu')
    print('⚠️ CPU mode（GPUなし）')

print(f'PyTorch: {torch.__version__}')
print(f'Device: {DEVICE}')

# ── 日本語フォント自動設定 ──
import matplotlib.font_manager as fm
import platform as _pf

def setup_japanese_font():
    _sys = _pf.system()
    candidates = {
        'Windows': ['Yu Gothic', 'MS Gothic', 'Meiryo', 'BIZ UDGothic'],
        'Darwin':  ['Hiragino Sans', 'Hiragino Kaku Gothic Pro'],
    }.get(_sys, ['Noto Sans CJK JP', 'IPAGothic', 'TakaoPGothic'])
    available = {f.name for f in fm.fontManager.ttflist}
    for fn in candidates:
        if fn in available:
            matplotlib.rcParams['font.family'] = fn
            break
    else:
        if _sys == 'Linux':
            os.system('apt-get install -y fonts-ipafont-gothic > /dev/null 2>&1')
            try:
                fm.fontManager.addfont('/usr/share/fonts/opentype/ipafont-gothic/ipag.ttf')
                matplotlib.rcParams['font.family'] = 'IPAGothic'
            except: pass
        matplotlib.rcParams['font.sans-serif'] = candidates + ['DejaVu Sans']
    matplotlib.rcParams['axes.unicode_minus'] = False

setup_japanese_font()

# ── カスタムカラーマップ（青→黄→赤）──
from matplotlib.colors import LinearSegmentedColormap

INSPECTION_CMAP = LinearSegmentedColormap.from_list(
    'inspection_heatmap',
    [(0.0, '#1a237e'), (0.15, '#1565c0'), (0.35, '#00bcd4'),
     (0.50, '#ffeb3b'), (0.70, '#ff9800'), (0.85, '#f44336'),
     (1.0, '#b71c1c')],
    N=256,
)

def create_overlay(base_img, error_map, alpha=0.5, cmap=None):
    """元画像にエラーマップをオーバーレイ合成"""
    if cmap is None: cmap = INSPECTION_CMAP
    if base_img.ndim == 2:
        base_rgb = np.stack([base_img]*3, axis=-1)
    else:
        base_rgb = base_img[:, :, :3]
    emax = error_map.max()
    error_norm = error_map / emax if emax > 0 else error_map
    heat_rgba = cmap(error_norm)[:, :, :3]
    return np.clip(base_rgb * (1-alpha) + heat_rgba * alpha, 0, 1)

print('✅ 日本語フォント + カスタムカラーマップ設定完了')


In [ ]:
# ── Google Drive マウント（結果保存用・任意）──
SAVE_TO_DRIVE = True  # Falseにするとローカル保存のみ

if SAVE_TO_DRIVE:
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        SAVE_DIR = Path('/content/drive/MyDrive/inspection_ai_results')
        SAVE_DIR.mkdir(parents=True, exist_ok=True)
        print(f'✅ 保存先: {SAVE_DIR}')
    except ImportError:
        # Windows/Mac ローカル実行時
        SAVE_DIR = Path('./results')
        SAVE_DIR.mkdir(parents=True, exist_ok=True)
        print(f'✅ ローカル保存: {SAVE_DIR}')
else:
    SAVE_DIR = Path('./results')
    SAVE_DIR.mkdir(parents=True, exist_ok=True)

## 1. モデル定義

### セクション1の解説: 2つのモデルのアーキテクチャ

#### Autoencoderの基本構造
Autoencoderは「Encoder（エンコーダ）」と「Decoder（デコーダ）」の2つのパーツで構成されます。

```
入力画像 (1×128×128)
   │
   ▼  Encoder: 畳み込み(Conv2d)で画像を圧縮
   │  128×128 → 64×64 → 32×32 → ... → 潜在変数(z)
   │
   ▼  Decoder: 逆畳み込み(ConvTranspose2d)で画像を復元
   │  潜在変数(z) → ... → 32×32 → 64×64 → 128×128
   ▼
再構成画像 (1×128×128)
```

#### Conv2d（畳み込み層）とは？
画像の局所的なパターン（エッジ、テクスチャなど）を抽出するフィルタです。
- `stride=2`: フィルタを2ピクセルずつずらすことで、画像サイズを半分に縮小
- `padding=1`: 画像の端にゼロを追加して、出力サイズを調整
- `BatchNorm2d`: 各層の出力を正規化し、学習を安定させるテクニック

#### Sparse AE vs Deep AE の違い

| 項目 | Sparse AE (SAE) | Deep AE (DAE) |
|------|-----------------|----------------|
| Encoder層数 | 2層 (Conv×2) | 4層 (Conv×8) |
| 潜在次元 | 64 | 128 |
| 正則化 | **L1正則化**（スパース化） | なし |
| 損失関数 | MSE + L1 | MSE + **SSIM** |
| パラメータ数 | 少ない（軽量） | 多い（高表現力） |

#### L1正則化（スパース化）とは？
潜在変数 `z` の絶対値の合計にペナルティをかけることで、**ほとんどのニューロンをゼロ近くに抑え、
少数のニューロンだけが強く反応する**状態を作ります。これにより：
- 重要な特徴だけに集中した、解釈しやすい表現が得られる
- モデルが過学習しにくくなる
- 計算量が削減され、エッジデバイスでの実行に適する

#### SSIM（構造類似度）損失とは？
人間の視覚に近い「構造の類似度」を評価する指標です。
MSE（ピクセル単位の差の二乗平均）だけでは捉えにくい「画像のぼやけ」や「構造の崩れ」も検出できます。
DAEではMSEに加えてSSIM損失を使うことで、より人間の感覚に近い再構成品質を実現しています。

In [ ]:
# ══════════════════════════════════════════════
# Sparse Autoencoder (SAE)
# L1正則化で潜在空間をスパース化
# ══════════════════════════════════════════════

class SparseAutoencoder(nn.Module):
    """
    Sparse Autoencoder
    - 軽量な2層Conv構成
    - L1正則化で少数の特徴量に集中
    - エッジ展開に適した小規模モデル
    """
    def __init__(self, in_channels=1, latent_dim=64):
        super().__init__()
        self.latent_dim = latent_dim
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 32, 4, stride=2, padding=1),
            nn.BatchNorm2d(32), nn.ReLU(True),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(True),
        )
        self.fc_encode = nn.Linear(64 * 32 * 32, latent_dim)
        self.fc_decode = nn.Linear(latent_dim, 64 * 32 * 32)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),
            nn.BatchNorm2d(32), nn.ReLU(True),
            nn.ConvTranspose2d(32, in_channels, 4, stride=2, padding=1),
            nn.Sigmoid(),
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_encode(h.view(h.size(0), -1))

    def decode(self, z):
        h = self.fc_decode(z).view(z.size(0), 64, 32, 32)
        return self.decoder(h)

    def forward(self, x):
        z = self.encode(x)
        return self.decode(z), z


class SparseAELoss(nn.Module):
    """MSE + L1スパース正則化"""
    def __init__(self, l1_weight=1e-3):
        super().__init__()
        self.mse = nn.MSELoss()
        self.l1_weight = l1_weight

    def forward(self, x, x_recon, z):
        recon = self.mse(x_recon, x)
        sparse = self.l1_weight * torch.mean(torch.abs(z))
        return recon + sparse, {'recon': recon.item(), 'sparse': sparse.item()}


print('✅ Sparse Autoencoder 定義完了')

In [ ]:
# ══════════════════════════════════════════════
# Deep Autoencoder (DAE)
# 多層Conv + SSIM損失による高精度モデル
# ══════════════════════════════════════════════

class DeepAutoencoder(nn.Module):
    """
    Deep Autoencoder
    - 4層Conv構成
    - 高い表現力で微細な異常も検出
    - パラメータ数はSAEの10倍以上
    """
    def __init__(self, in_channels=1, latent_dim=128):
        super().__init__()
        self.enc1 = self._conv(in_channels, 32)
        self.enc2 = self._conv(32, 64)
        self.enc3 = self._conv(64, 128)
        self.enc4 = self._conv(128, 256)
        self.fc_encode = nn.Linear(256 * 8 * 8, latent_dim)
        self.fc_decode = nn.Linear(latent_dim, 256 * 8 * 8)
        self.dec4 = self._deconv(256, 128)
        self.dec3 = self._deconv(128, 64)
        self.dec2 = self._deconv(64, 32)
        self.dec1 = nn.Sequential(
            nn.ConvTranspose2d(32, in_channels, 4, stride=2, padding=1),
            nn.Sigmoid(),
        )

    @staticmethod
    def _conv(inc, outc):
        return nn.Sequential(
            nn.Conv2d(inc, outc, 4, 2, 1), nn.BatchNorm2d(outc), nn.LeakyReLU(0.2, True),
            nn.Conv2d(outc, outc, 3, 1, 1), nn.BatchNorm2d(outc), nn.LeakyReLU(0.2, True),
        )

    @staticmethod
    def _deconv(inc, outc):
        return nn.Sequential(
            nn.ConvTranspose2d(inc, outc, 4, 2, 1), nn.BatchNorm2d(outc), nn.ReLU(True),
            nn.Conv2d(outc, outc, 3, 1, 1), nn.BatchNorm2d(outc), nn.ReLU(True),
        )

    def encode(self, x):
        h = self.enc4(self.enc3(self.enc2(self.enc1(x))))
        return self.fc_encode(h.view(h.size(0), -1))

    def decode(self, z):
        h = self.fc_decode(z).view(z.size(0), 256, 8, 8)
        return self.dec1(self.dec2(self.dec3(self.dec4(h))))

    def forward(self, x):
        z = self.encode(x)
        return self.decode(z), z


class DeepAELoss(nn.Module):
    """MSE + 簡易SSIM構造損失"""
    def __init__(self, ssim_weight=0.1):
        super().__init__()
        self.mse = nn.MSELoss()
        self.ssim_weight = ssim_weight

    def _ssim_approx(self, x, y, ws=11):
        C1, C2 = 0.01**2, 0.03**2
        pad = ws // 2
        mu_x = nn.functional.avg_pool2d(x, ws, 1, pad)
        mu_y = nn.functional.avg_pool2d(y, ws, 1, pad)
        s_x = nn.functional.avg_pool2d(x**2, ws, 1, pad) - mu_x**2
        s_y = nn.functional.avg_pool2d(y**2, ws, 1, pad) - mu_y**2
        s_xy = nn.functional.avg_pool2d(x*y, ws, 1, pad) - mu_x*mu_y
        return ((2*mu_x*mu_y+C1)*(2*s_xy+C2) / ((mu_x**2+mu_y**2+C1)*(s_x+s_y+C2))).mean()

    def forward(self, x, x_recon, z=None):
        recon = self.mse(x_recon, x)
        ssim = 1.0 - self._ssim_approx(x, x_recon)
        return recon + self.ssim_weight * ssim, {'recon': recon.item(), 'ssim': ssim.item()}


print('✅ Deep Autoencoder 定義完了')

## 2. 合成データ生成

### セクション2の解説: 合成データはなぜ使うのか

#### 合成データ（Synthetic Data）とは？
実際の製品画像の代わりに、プログラムで人工的に生成したテスト用画像です。
このノートブックでは以下を生成しています：

**正常品画像の構成要素:**
- ベース: 均一なグレー（明るさ0.6）に微小なノイズを加えたもの
- 格子パターン: 16ピクセル間隔の細い線（基板や表面テクスチャを模擬）
- 円形特徴: 中央付近の円形マスク（部品の形状を模擬）

**4種類の欠陥パターン:**
| 欠陥タイプ | 英名 | 特徴 | 実際の例 |
|-----------|------|------|---------|
| 傷 | scratch | 直線状の暗い筋 | 表面のひっかき傷 |
| 汚れ | stain | ガウシアン分布の暗い斑点 | 油汚れ、指紋 |
| 欠け | missing | 円形の暗い穴 | 部品の欠落、穴 |
| 変色 | discolor | 矩形の明るい領域 | 焼けムラ、変色 |

#### なぜ合成データを使うのか？
1. **手軽に動作検証**できる（実データ不要で即座に試せる）
2. **欠陥の種類と位置が既知**なので、検出精度を正確に評価できる
3. 実データの準備ができたら、セクション9のテンプレートで簡単に差し替え可能

#### PyTorchの`Dataset`と`DataLoader`
- `Dataset`: データ1件の取得方法を定義するクラス。`__getitem__`で画像・ラベル・マスクを返す
- `DataLoader`: Datasetからミニバッチ（複数データのまとまり）を自動生成する。`batch_size=16`なら16枚ずつモデルに渡す

In [ ]:
# ══════════════════════════════════════════════
# 合成テストデータ
# 正常品 + 4種類の欠陥（scratch/stain/missing/discolor）
# ══════════════════════════════════════════════

def generate_normal_image(size=128, seed=None):
    """正常品画像（均一テクスチャ + 格子パターン + 円形特徴）"""
    if seed is not None:
        np.random.seed(seed)
    img = np.ones((size, size), dtype=np.float32) * 0.6
    img += np.random.normal(0, 0.02, (size, size)).astype(np.float32)
    for i in range(0, size, 16):
        img[i:i+1, :] += 0.05
        img[:, i:i+1] += 0.05
    y, x = np.ogrid[-size//2:size//2, -size//2:size//2]
    r = np.sqrt(x*x + y*y).astype(np.float32)
    mask = (r < size * 0.35).astype(np.float32)
    img = img * (1 - mask * 0.1) + mask * 0.05
    return np.clip(img, 0, 1)


def add_defect(img, defect_type='scratch', seed=None):
    """欠陥を追加 → (異常画像, マスク)"""
    if seed is not None:
        np.random.seed(seed)
    size = img.shape[0]
    out = img.copy()
    mask = np.zeros_like(img)

    if defect_type == 'scratch':
        ys, xs = np.random.randint(20, size-20), np.random.randint(20, size-40)
        length, angle = np.random.randint(30, 60), np.random.uniform(-0.3, 0.3)
        for i in range(length):
            yi, xi = int(ys + i*np.sin(angle)), int(xs + i*np.cos(angle))
            if 0 <= yi < size and 0 <= xi < size:
                w = np.random.randint(1, 3)
                out[max(0,yi-w):yi+w, max(0,xi-w):xi+w] -= 0.3
                mask[max(0,yi-w):yi+w, max(0,xi-w):xi+w] = 1.0
    elif defect_type == 'stain':
        cy, cx = np.random.randint(30, size-30, 2)
        rad = np.random.randint(8, 20)
        yy, xx = np.ogrid[:size, :size]
        blob = np.exp(-((xx-cx)**2 + (yy-cy)**2) / (2*rad**2))
        out -= blob.astype(np.float32) * 0.25
        mask = (blob > 0.3).astype(np.float32)
    elif defect_type == 'missing':
        cy, cx = np.random.randint(30, size-30, 2)
        rad = np.random.randint(5, 15)
        yy, xx = np.ogrid[:size, :size]
        hole = (np.sqrt((xx-cx)**2 + (yy-cy)**2) < rad).astype(np.float32)
        out = out * (1 - hole) + hole * 0.1
        mask = hole
    elif defect_type == 'discolor':
        y1, x1 = np.random.randint(10, size-40), np.random.randint(10, size-40)
        h, w = np.random.randint(15, 35), np.random.randint(15, 35)
        out[y1:y1+h, x1:x1+w] += 0.15
        mask[y1:y1+h, x1:x1+w] = 1.0

    return np.clip(out, 0, 1), mask


class InspectionDataset(Dataset):
    def __init__(self, n_normal=200, n_anomaly=50, image_size=128, include_anomaly=True):
        self.images, self.labels, self.masks, self.types = [], [], [], []
        for i in range(n_normal):
            img = generate_normal_image(image_size, seed=i)
            self.images.append(img); self.labels.append(0)
            self.masks.append(np.zeros_like(img)); self.types.append('normal')
        if include_anomaly:
            dtypes = ['scratch', 'stain', 'missing', 'discolor']
            for i in range(n_anomaly):
                base = generate_normal_image(image_size, seed=1000+i)
                dt = dtypes[i % 4]
                defect, m = add_defect(base, dt, seed=2000+i)
                self.images.append(defect); self.labels.append(1)
                self.masks.append(m); self.types.append(dt)

    def __len__(self): return len(self.images)
    def __getitem__(self, idx):
        return (torch.FloatTensor(self.images[idx]).unsqueeze(0),
                self.labels[idx],
                torch.FloatTensor(self.masks[idx]).unsqueeze(0))


print('✅ データ生成モジュール定義完了')

In [ ]:
# ── サンプル画像の確認 ──
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('合成データサンプル', fontsize=16, fontweight='bold')

for i in range(5):
    img = generate_normal_image(128, seed=i)
    axes[0, i].imshow(img, cmap='gray', vmin=0, vmax=1)
    axes[0, i].set_title(f'正常品 #{i+1}', fontsize=10)
    axes[0, i].axis('off')

for i, dt in enumerate(['scratch', 'stain', 'missing', 'discolor', 'scratch']):
    base = generate_normal_image(128, seed=100+i)
    defect, _ = add_defect(base, dt, seed=200+i)
    axes[1, i].imshow(defect, cmap='gray', vmin=0, vmax=1)
    axes[1, i].set_title(f'異常: {dt}', fontsize=10, color='red')
    axes[1, i].axis('off')

plt.tight_layout()
plt.savefig(SAVE_DIR / 'sample_data.png', dpi=150, bbox_inches='tight')
plt.show()
plt.close()
print(f'💾 保存: {SAVE_DIR}/sample_data.png')

## 3. 検証パラメータ設定

ここでパラメータを調整して、異なる条件での比較が可能です。

### セクション3の解説: パラメータの意味と調整の指針

各パラメータがモデルの動作にどう影響するかを解説します。

#### データ関連パラメータ
| パラメータ | デフォルト | 説明 |
|-----------|----------|------|
| `n_train` | 200 | 学習に使う正常画像の枚数。増やすほどモデルが「正常」を正確に学ぶが、学習に時間がかかる |
| `n_test_normal` | 50 | テスト用の正常画像数 |
| `n_test_anomaly` | 50 | テスト用の異常画像数 |
| `image_size` | 128 | 画像の縦横ピクセル数。64にすると4倍高速になるが解像度が落ちる |
| `batch_size` | 16 | 一度にモデルに入力する画像枚数。GPUメモリに余裕があれば増やせる |

#### モデル固有パラメータ
| パラメータ | 説明 | 調整のコツ |
|-----------|------|-----------|
| `sae_latent_dim` | SAEの潜在次元数。少ないほど圧縮が強い | 32〜128の範囲で試す |
| `sae_l1_weight` | L1正則化の強さ(λ)。大きいほどスパース化が強い | 1e-4〜1e-2で調整。大きすぎると再構成が崩れる |
| `dae_latent_dim` | DAEの潜在次元数 | SAEより大きくして表現力を確保 |
| `dae_ssim_weight` | SSIM損失の重み。大きいほど構造保存を重視 | 0.05〜0.3で調整 |

#### 学習パラメータ
| パラメータ | 説明 |
|-----------|------|
| `n_epochs` | 学習の繰り返し回数。学習曲線が収束するまで増やす |
| `learning_rate` | 学習率。パラメータ更新の大きさ。大きすぎると発散、小さすぎると収束が遅い |

In [ ]:
# ╔══════════════════════════════════════════════╗
# ║  🎛️ 検証パラメータ（ここを変えて実験）       ║
# ╚══════════════════════════════════════════════╝

CONFIG = {
    # データ設定
    'n_train': 200,           # 学習データ数（正常品）
    'n_test_normal': 50,      # テスト正常品数
    'n_test_anomaly': 50,     # テスト異常品数
    'image_size': 128,        # 画像サイズ (64 or 128)
    'batch_size': 16,

    # Sparse AE 設定
    'sae_latent_dim': 64,     # 潜在次元
    'sae_l1_weight': 1e-3,    # L1正則化強度 (λ)

    # Deep AE 設定
    'dae_latent_dim': 128,    # 潜在次元
    'dae_ssim_weight': 0.1,   # SSIM損失重み

    # 学習設定
    'n_epochs': 30,
    'learning_rate': 1e-3,
}

print('📋 検証パラメータ:')
for k, v in CONFIG.items():
    print(f'  {k}: {v}')

## 4. データ準備

### セクション4の解説: DataLoaderの役割

#### なぜ学習データに正常品「だけ」を使うのか？
Autoencoderによる異常検知では、モデルに「正常とは何か」だけを覚えさせます。
テスト時に正常から外れた画像（＝異常品）がうまく再構成できないことを利用して異常を検出します。

そのため：
- **学習データ** (`train_dataset`): 正常品のみ200枚（`include_anomaly=False`）
- **テストデータ** (`test_dataset`): 正常品50枚 + 異常品50枚（性能を測るために両方必要）

#### DataLoaderの設定
- `shuffle=True`（学習時）: データの順番をランダムにして、特定の順序への過適合を防ぐ
- `shuffle=False`（テスト時）: 結果の再現性のため、順番は固定

In [ ]:
# ── DataLoader作成 ──
train_dataset = InspectionDataset(
    n_normal=CONFIG['n_train'], n_anomaly=0,
    image_size=CONFIG['image_size'], include_anomaly=False,
)
test_dataset = InspectionDataset(
    n_normal=CONFIG['n_test_normal'], n_anomaly=CONFIG['n_test_anomaly'],
    image_size=CONFIG['image_size'], include_anomaly=True,
)

train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False)

print(f'✅ 学習データ: {len(train_dataset)}枚（正常品のみ）')
print(f'✅ テストデータ: {len(test_dataset)}枚（正常{CONFIG["n_test_normal"]} + 異常{CONFIG["n_test_anomaly"]}）')

## 5. モデル学習

### セクション5の解説: モデル学習の流れ

#### 学習ループの全体像
```
for epoch in range(30):           # 全データを30回繰り返す
    for batch in train_loader:    # 16枚ずつ処理
        1. 画像をモデルに入力 → 再構成画像と潜在変数を得る
        2. 損失（Loss）を計算 → 入力と出力の「ズレ」を数値化
        3. 誤差逆伝播（Backpropagation） → 損失を小さくする方向を計算
        4. パラメータ更新（Optimizer） → モデルの重みを微調整
```

#### 主要コンポーネントの役割
| コンポーネント | 説明 |
|-------------|------|
| **Loss関数** | 「どれだけ再構成に失敗したか」を数値化する関数 |
| **Optimizer (Adam)** | 損失を減らすようにモデルのパラメータを更新するアルゴリズム |
| **Scheduler (CosineAnnealing)** | 学習率をコサインカーブに沿って徐々に減らす。序盤は大きく学習し、終盤は微調整する |

#### 損失関数の違い
- **SAE**: `MSE + λ × L1(z)` → 再構成精度 + スパース性のバランス
- **DAE**: `MSE + α × (1 - SSIM)` → 再構成精度 + 構造保存のバランス

#### 学習曲線の読み方
- Lossが下がっていれば学習が進んでいる
- 途中で横ばいになれば「収束」→ これ以上エポック数を増やしても改善しにくい
- 急に上昇したら「発散」→ 学習率が大きすぎる可能性

#### ユーティリティ関数
- `count_params()`: モデルの総パラメータ数を数える
- `model_size_mb()`: モデルのメモリ使用量をMB単位で計算
- `sparsity()`: 潜在変数のうち、ほぼゼロ（< 0.01）のニューロンの割合を計算

In [ ]:
# ── 学習関数 ──
def train_model(model, criterion, loader, config, model_name):
    """モデル学習（プログレスバー付き）"""
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config['n_epochs'])
    model.to(DEVICE)
    model.train()
    losses = []
    t0 = time.perf_counter()

    for epoch in range(config['n_epochs']):
        ep_loss, nb = 0.0, 0
        for imgs, _, _ in loader:
            imgs = imgs.to(DEVICE)
            optimizer.zero_grad()
            recon, z = model(imgs)
            loss, _ = criterion(imgs, recon, z)
            loss.backward()
            optimizer.step()
            ep_loss += loss.item(); nb += 1
        scheduler.step()
        avg = ep_loss / max(nb, 1)
        losses.append(avg)
        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f'  {model_name} Epoch {epoch+1:3d}/{config["n_epochs"]} | Loss: {avg:.6f}')

    train_time = time.perf_counter() - t0
    print(f'  ✅ {model_name} 学習完了: {train_time:.1f}秒')
    return losses, train_time


# ── ユーティリティ ──
def count_params(model):
    return sum(p.numel() for p in model.parameters())

def model_size_mb(model):
    ps = sum(p.nelement() * p.element_size() for p in model.parameters())
    bs = sum(b.nelement() * b.element_size() for b in model.buffers())
    return (ps + bs) / (1024 * 1024)

def sparsity(z, thr=0.01):
    return (z.abs() < thr).float().mean().item()

In [ ]:
# ── モデル初期化 ──
sae = SparseAutoencoder(in_channels=1, latent_dim=CONFIG['sae_latent_dim'])
dae = DeepAutoencoder(in_channels=1, latent_dim=CONFIG['dae_latent_dim'])
sae_criterion = SparseAELoss(l1_weight=CONFIG['sae_l1_weight'])
dae_criterion = DeepAELoss(ssim_weight=CONFIG['dae_ssim_weight'])

print('┌────────────────────────────────────────┐')
print(f'│ SAE: {count_params(sae):>10,} params | {model_size_mb(sae):>6.2f} MB │')
print(f'│ DAE: {count_params(dae):>10,} params | {model_size_mb(dae):>6.2f} MB │')
print('└────────────────────────────────────────┘')

In [ ]:
# ══════════════════════════════════════════════
# 🏋️ Sparse AE 学習
# ══════════════════════════════════════════════
print('━' * 50)
print('🏋️ Sparse Autoencoder 学習開始')
print('━' * 50)
sae_losses, sae_train_time = train_model(sae, sae_criterion, train_loader, CONFIG, 'SAE')

In [ ]:
# ══════════════════════════════════════════════
# 🏋️ Deep AE 学習
# ══════════════════════════════════════════════
print('━' * 50)
print('🏋️ Deep Autoencoder 学習開始')
print('━' * 50)
dae_losses, dae_train_time = train_model(dae, dae_criterion, train_loader, CONFIG, 'DAE')

In [ ]:
# ── 学習曲線比較 ──
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(sae_losses, color='#22d3ee', linewidth=2, label=f'Sparse AE ({sae_train_time:.1f}s)')
ax.plot(dae_losses, color='#f97316', linewidth=2, label=f'Deep AE ({dae_train_time:.1f}s)')
ax.set_xlabel('Epoch'); ax.set_ylabel('Loss')
ax.set_title('学習曲線比較', fontsize=14, fontweight='bold')
ax.legend(); ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(SAVE_DIR / 'training_curves.png', dpi=150, bbox_inches='tight')
plt.show()
plt.close()

## 6. 評価

### セクション6の解説: 評価指標の意味

異常検知モデルの性能を測る指標を解説します。

#### 異常スコアとは？
各テスト画像に対して「入力と再構成画像の差（MSE）」を計算した値です。
- スコアが**小さい** → モデルがうまく再構成できた → 正常品らしい
- スコアが**大きい** → モデルがうまく再構成できなかった → 異常品らしい

#### 評価指標一覧
| 指標 | 意味 | 理想値 | 初心者向け解説 |
|------|------|--------|---------------|
| **AUC-ROC** | 正常/異常の分離能力（0〜1） | 1.0 | 「1に近いほど優秀」と覚えればOK。0.5はランダム（使い物にならない） |
| **Best F1** | 最適閾値でのF1スコア | 1.0 | 適合率（Precision）と再現率（Recall）の調和平均。バランスの良さを示す |
| **F1 (3σ閾値)** | 統計的閾値（平均+3σ）でのF1 | 1.0 | 実運用で使いやすい自動閾値での性能 |
| **FPR@95TPR** | 異常品の95%を検出した時の誤検知率 | 0.0 | 「異常を95%検出する設定で、正常品を何%間違えるか」 |
| **推論速度** | 1枚の画像を処理するのにかかる時間 | 小さい | リアルタイム検査では1ms以下が理想 |

#### 3σ閾値とは？
正常品のスコア分布から「平均 + 3×標準偏差」を閾値に設定する方法です。
正規分布を仮定すると、正常品がこの閾値を超える確率は約0.3%で、実用的な自動設定として広く使われます。

#### スコア分布グラフの見方
- **緑のヒストグラム**: 正常品のスコア分布
- **赤のヒストグラム**: 異常品のスコア分布
- **白い破線**: 3σ閾値

緑と赤が離れているほど「正常と異常をはっきり区別できている」ことを意味します。
重なりが多いと誤検知や見逃しが増えます。

In [ ]:
# ── 異常スコア計算 ──
def compute_scores(model, loader, device):
    """全テストデータの異常スコアを計算"""
    model.eval()
    scores, labels, error_maps = [], [], []
    total_time, n = 0.0, 0
    with torch.no_grad():
        for imgs, lbl, _ in loader:
            imgs = imgs.to(device)
            bs = imgs.size(0)
            t0 = time.perf_counter()
            recon, z = model(imgs)
            if device.type == 'cuda': torch.cuda.synchronize()
            total_time += time.perf_counter() - t0
            n += bs
            err = (imgs - recon) ** 2
            scores.extend(err.mean(dim=(1,2,3)).cpu().numpy())
            labels.extend(lbl.numpy())
            error_maps.extend(err.squeeze(1).cpu().numpy())
    return np.array(scores), np.array(labels), error_maps, (total_time/n)*1000


def compute_metrics(scores, labels):
    """評価指標"""
    m = {}
    if len(np.unique(labels)) > 1:
        m['auc_roc'] = roc_auc_score(labels, scores)
        fpr, tpr, thr = roc_curve(labels, scores)
        idx95 = np.argmin(np.abs(tpr - 0.95))
        m['fpr_at_95tpr'] = fpr[idx95]
        prec, rec, pthr = precision_recall_curve(labels, scores)
        f1s = 2 * prec * rec / (prec + rec + 1e-8)
        m['best_f1'] = f1s.max()
        ns = scores[labels == 0]
        thr3s = ns.mean() + 3 * ns.std()
        m['f1_3sigma'] = f1_score(labels, (scores > thr3s).astype(int), zero_division=0)
    m['normal_mean'] = scores[labels==0].mean()
    m['normal_std'] = scores[labels==0].std()
    m['anomaly_mean'] = scores[labels==1].mean()
    m['anomaly_std'] = scores[labels==1].std()
    return m


print('📊 異常スコア計算中...')
sae_scores, sae_labels, sae_maps, sae_inf = compute_scores(sae, test_loader, DEVICE)
dae_scores, dae_labels, dae_maps, dae_inf = compute_scores(dae, test_loader, DEVICE)
sae_m = compute_metrics(sae_scores, sae_labels)
dae_m = compute_metrics(dae_scores, dae_labels)
print('✅ 完了')

In [ ]:
# ══════════════════════════════════════════════
# 📊 定量評価結果
# ══════════════════════════════════════════════

print('\n' + '═' * 65)
print('  📊 定量評価結果')
print('═' * 65)
print(f'{"指標":<20} {"Sparse AE":>15} {"Deep AE":>15}  {"Winner":>8}')
print('─' * 65)

comparisons = [
    ('AUC-ROC',          sae_m.get('auc_roc',0),     dae_m.get('auc_roc',0),     'higher'),
    ('Best F1',          sae_m.get('best_f1',0),     dae_m.get('best_f1',0),     'higher'),
    ('F1 (3σ閾値)',      sae_m.get('f1_3sigma',0),   dae_m.get('f1_3sigma',0),   'higher'),
    ('FPR@95TPR',        sae_m.get('fpr_at_95tpr',0),dae_m.get('fpr_at_95tpr',0),'lower'),
    ('推論速度 (ms)',    sae_inf,                     dae_inf,                     'lower'),
    ('学習時間 (s)',     sae_train_time,              dae_train_time,              'lower'),
    ('パラメータ数',    count_params(sae),           count_params(dae),           'lower'),
    ('モデルサイズ(MB)', model_size_mb(sae),          model_size_mb(dae),          'lower'),
]

for name, sv, dv, direction in comparisons:
    if isinstance(sv, int):
        sf, df = f'{sv:>15,}', f'{dv:>15,}'
    else:
        sf, df = f'{sv:>15.4f}', f'{dv:>15.4f}'
    if direction == 'higher':
        w = '  SAE ✓' if sv > dv else '  DAE ✓'
    else:
        w = '  SAE ✓' if sv < dv else '  DAE ✓'
    print(f'{name:<20} {sf} {df} {w}')

print('═' * 65)

# 精度差の評価
auc_diff = abs(sae_m.get('auc_roc',0) - dae_m.get('auc_roc',0))
speed_ratio = dae_inf / max(sae_inf, 0.001)
print(f'\n📌 AUC-ROC差: {auc_diff:.4f}  |  速度比: SAEが{speed_ratio:.1f}倍高速')
if auc_diff < 0.05 and speed_ratio > 3:
    print('💡 結論: 精度差が小さく速度差が大きい → スパースモデルが実用的')
elif auc_diff > 0.1:
    print('💡 結論: 精度差が大きい → ディープモデルを推奨')
else:
    print('💡 結論: トレードオフあり → 製造ライン要件に応じて選択')

In [ ]:
# ── 異常スコア分布 ──
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for ax, scores, labels, title, color in [
    (ax1, sae_scores, sae_labels, 'Sparse AE', '#22d3ee'),
    (ax2, dae_scores, dae_labels, 'Deep AE', '#f97316'),
]:
    ns, ans = scores[labels==0], scores[labels==1]
    ax.hist(ns, bins=30, alpha=0.7, color='#10b981', label='正常', density=True)
    ax.hist(ans, bins=30, alpha=0.7, color='#ef4444', label='異常', density=True)
    ax.axvline(ns.mean() + 3*ns.std(), color='white', ls='--', lw=1.5, label='3σ閾値')
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.set_xlabel('異常スコア'); ax.legend()
    ax.grid(alpha=0.2)

plt.suptitle('異常スコア分布比較', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.savefig(SAVE_DIR / 'score_distribution.png', dpi=150, bbox_inches='tight')
plt.show()
plt.close()

## 7. ヒートマップ可視化

### セクション7の解説: ヒートマップの見方

#### ヒートマップとは？
画像上の各ピクセルで「どれだけ再構成に失敗したか（＝異常度）」を色で表示したものです。

- **青色**: 再構成誤差が小さい（正常な箇所）
- **黄色**: 中程度の誤差（やや気になる箇所）
- **赤色**: 再構成誤差が大きい（異常の可能性が高い箇所）

#### 各列の意味
| 列 | 表示内容 | 説明 |
|----|---------|------|
| 元画像 | テスト画像そのもの | 入力として与えた画像 |
| 真値マスク | 実際の欠陥位置 | 合成データでは正解がわかっているので比較に使う |
| SAE再構成 | SAEが復元した画像 | 正常な部分はうまく復元され、異常部分はぼやける |
| DAE再構成 | DAEが復元した画像 | SAEより高精度だが、処理は遅い |
| SAEエラー / DAEエラー | 再構成誤差マップ | `|入力 - 再構成|` の絶対値。赤い箇所が異常候補 |
| SAEオーバーレイ | 元画像にエラーを重ねた図 | 実際の検査画面で使うイメージ |

#### 潜在空間のスパース度
棒グラフは潜在変数 `z` の各ニューロンの活性化値（絶対値）を示しています。
- **SAE**: L1正則化により、大部分のニューロンがゼロ近くに抑えられ、少数だけが高い値を持つ（**スパース**）
- **DAE**: 正則化なしのため、多くのニューロンがまんべんなく活性化している（**密**）

スパースな表現は「この画像のどの特徴が重要か」が明確で、解釈性が高いという利点があります。

In [ ]:
# ── 異常検知ヒートマップ比較 ──
all_imgs, all_labels_t, all_masks_t = [], [], []
for imgs, lbl, msk in test_loader:
    all_imgs.append(imgs); all_labels_t.append(lbl); all_masks_t.append(msk)
all_imgs = torch.cat(all_imgs)
all_labels = torch.cat(all_labels_t).numpy()
all_masks = torch.cat(all_masks_t)

anomaly_idx = np.where(all_labels == 1)[0]
n_show = min(6, len(anomaly_idx))
sel = anomaly_idx[:n_show]

sae.eval(); dae.eval()
n_cols = 7
fig, axes = plt.subplots(n_show, n_cols, figsize=(n_cols*3, n_show*3))
if n_show == 1: axes = axes.reshape(1, -1)

col_titles = ['元画像', '真値マスク', 'SAE再構成', 'DAE再構成',
              'SAEエラー', 'DAEエラー', 'SAEオーバーレイ']
for j, t in enumerate(col_titles):
    axes[0, j].set_title(t, fontsize=10, fontweight='bold')

for i, idx in enumerate(sel):
    img = all_imgs[idx:idx+1].to(DEVICE)
    mask = all_masks[idx].squeeze().numpy()
    with torch.no_grad():
        sr, _ = sae(img)
        dr, _ = dae(img)
    inp = all_imgs[idx].squeeze().numpy()
    sr_np = sr.squeeze().cpu().numpy()
    dr_np = dr.squeeze().cpu().numpy()
    se = np.abs(inp - sr_np)
    de = np.abs(inp - dr_np)
    overlay = create_overlay(inp, se, alpha=0.55)

    panels = [
        (inp,     'gray', 1.0,  False),
        (mask,    'Reds', 1.0,  False),
        (sr_np,   'gray', 1.0,  False),
        (dr_np,   'gray', 1.0,  False),
        (se,      INSPECTION_CMAP, None, True),
        (de,      INSPECTION_CMAP, None, True),
        (overlay, None,   None, False),
    ]
    for j, (p, cm, vmax_f, add_cbar) in enumerate(panels):
        ax = axes[i, j]
        if cm is None:
            ax.imshow(p)
        elif vmax_f is not None:
            ax.imshow(p, cmap=cm, vmin=0, vmax=vmax_f)
        else:
            vmax = max(p.max(), 0.001)
            im = ax.imshow(p, cmap=cm, vmin=0, vmax=vmax)
            if add_cbar:
                cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
                cbar.ax.tick_params(labelsize=6)
        ax.axis('off')

plt.suptitle('異常検知ヒートマップ比較', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.savefig(SAVE_DIR / 'heatmap_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
plt.close()


In [ ]:
# ── 潜在空間のスパース度可視化 ──
sample = all_imgs[:1].to(DEVICE)
with torch.no_grad():
    _, sz = sae(sample)
    _, dz = dae(sample)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 3.5))

szn = sz.cpu().numpy().flatten()
dzn = dz.cpu().numpy().flatten()

ax1.bar(range(len(szn)), np.abs(szn), color='#22d3ee', alpha=0.8, width=1.0)
ax1.set_title(f'SAE 潜在表現 (スパース度: {sparsity(sz):.1%})', fontweight='bold')
ax1.set_xlabel('ニューロンIndex'); ax1.set_ylabel('|活性化値|')

ax2.bar(range(len(dzn)), np.abs(dzn), color='#f97316', alpha=0.8, width=1.0)
ax2.set_title(f'DAE 潜在表現 (スパース度: {sparsity(dz):.1%})', fontweight='bold')
ax2.set_xlabel('ニューロンIndex'); ax2.set_ylabel('|活性化値|')

plt.tight_layout()
plt.savefig(SAVE_DIR / 'sparsity_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
plt.close()
print(f'💡 SAEは少数のニューロンだけが強く活性化 → 重要な特徴に集中')

## 8. モデル保存 & エクスポート

### セクション8の解説: モデルの保存形式

#### `.pth`ファイル（PyTorch形式）
PyTorchの標準的な保存形式です。`state_dict()`でモデルの重み（パラメータ）だけを保存します。
読み込む際はモデルクラスの定義が必要です。Python + PyTorch環境での開発・研究に使います。

#### `.onnx`ファイル（ONNX形式）
**ONNX (Open Neural Network Exchange)** は、異なるフレームワーク間でモデルを共有するための標準形式です。
- C++やRustなどのプログラムからPythonなしで実行できる
- NVIDIA TensorRT、Intel OpenVINOなどの推論エンジンで高速化できる
- エッジデバイス（Jetson、Raspberry Piなど）にデプロイする際に必須

#### `benchmark_results.json`
すべての評価結果（パラメータ設定、各モデルのスコア、推論速度など）をJSON形式で保存します。
後から結果を比較・分析したり、レポートを作成する際に便利です。

In [ ]:
# ── モデル保存 ──
torch.save(sae.state_dict(), SAVE_DIR / 'sparse_ae.pth')
torch.save(dae.state_dict(), SAVE_DIR / 'deep_ae.pth')
print(f'💾 モデル保存: {SAVE_DIR}')

# ── ONNX エクスポート（エッジ展開用）──
try:
    dummy = torch.randn(1, 1, CONFIG['image_size'], CONFIG['image_size']).to('cpu')
    sae_cpu = SparseAutoencoder(latent_dim=CONFIG['sae_latent_dim']).cpu()
    sae_cpu.load_state_dict(sae.cpu().state_dict())
    sae_cpu.eval()
    torch.onnx.export(
        sae_cpu, dummy,
        str(SAVE_DIR / 'sparse_ae.onnx'),
        input_names=['input'], output_names=['reconstruction', 'latent'],
        dynamic_axes={'input': {0: 'batch'}, 'reconstruction': {0: 'batch'}},
        opset_version=14,
    )
    onnx_size = os.path.getsize(SAVE_DIR / 'sparse_ae.onnx') / (1024*1024)
    print(f'✅ ONNX エクスポート完了: sparse_ae.onnx ({onnx_size:.2f} MB)')
    sae.to(DEVICE)  # GPUに戻す
except Exception as e:
    print(f'⚠️ ONNX エクスポートスキップ: {e}')

In [ ]:
# ── 評価結果をJSONで保存 ──
results = {
    'config': CONFIG,
    'device': str(DEVICE),
    'sparse_ae': {
        'params': count_params(sae),
        'size_mb': round(model_size_mb(sae), 3),
        'train_time_s': round(sae_train_time, 2),
        'inference_ms': round(sae_inf, 3),
        'metrics': {k: round(float(v), 5) for k, v in sae_m.items()},
    },
    'deep_ae': {
        'params': count_params(dae),
        'size_mb': round(model_size_mb(dae), 3),
        'train_time_s': round(dae_train_time, 2),
        'inference_ms': round(dae_inf, 3),
        'metrics': {k: round(float(v), 5) for k, v in dae_m.items()},
    },
}

with open(SAVE_DIR / 'benchmark_results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f'💾 結果保存: {SAVE_DIR}/benchmark_results.json')
print('\n📂 保存ファイル一覧:')
for f in sorted(SAVE_DIR.iterdir()):
    sz = f.stat().st_size / 1024
    print(f'  {f.name:<30} {sz:>8.1f} KB')

## 9. 実データへの切り替え方法

合成データで動作確認ができたら、以下のセルを使って実データに差し替えられます。

In [ ]:
# ══════════════════════════════════════════════
# 実データ読み込みテンプレート
# Google Drive または ローカルフォルダから画像を読み込む
# ══════════════════════════════════════════════

# --- このセルは必要な時にコメントを外して使用 ---

# from torchvision import transforms
# from PIL import Image
# import glob
#
# class RealImageDataset(Dataset):
#     """実画像データセット"""
#     def __init__(self, image_dir, label=0, image_size=128):
#         self.paths = sorted(glob.glob(f'{image_dir}/*.png') +
#                            glob.glob(f'{image_dir}/*.jpg') +
#                            glob.glob(f'{image_dir}/*.bmp'))
#         self.label = label
#         self.transform = transforms.Compose([
#             transforms.Resize((image_size, image_size)),
#             transforms.Grayscale(num_output_channels=1),
#             transforms.ToTensor(),
#         ])
#
#     def __len__(self): return len(self.paths)
#     def __getitem__(self, idx):
#         img = Image.open(self.paths[idx]).convert('L')
#         img_t = self.transform(img)
#         mask = torch.zeros_like(img_t)
#         return img_t, self.label, mask
#
# # 使い方:
# # Google Drive上にフォルダを作成:
# #   /content/drive/MyDrive/inspection_data/
# #     ├── train/normal/     ← 正常品画像
# #     └── test/
# #         ├── normal/       ← テスト正常品
# #         └── anomaly/      ← テスト異常品
#
# REAL_DATA_DIR = '/content/drive/MyDrive/inspection_data'
#
# train_real = RealImageDataset(f'{REAL_DATA_DIR}/train/normal', label=0)
# test_normal = RealImageDataset(f'{REAL_DATA_DIR}/test/normal', label=0)
# test_anomaly = RealImageDataset(f'{REAL_DATA_DIR}/test/anomaly', label=1)
# test_real = torch.utils.data.ConcatDataset([test_normal, test_anomaly])
#
# train_loader = DataLoader(train_real, batch_size=16, shuffle=True)
# test_loader = DataLoader(test_real, batch_size=16, shuffle=False)
#
# print(f'実データ: 学習{len(train_real)}枚, テスト{len(test_real)}枚')

print('💡 実データ使用時はコメントを外して実行してください')
print('   フォルダ構成:')
print('   inspection_data/')
print('   ├── train/normal/     ← 正常品画像')
print('   └── test/')
print('       ├── normal/       ← テスト正常品')
print('       └── anomaly/      ← テスト異常品')

---
## 📝 補足情報

### Windows ローカル実行方法

```bash
# 1. このノートブックをダウンロード
# 2. Jupyter Notebook/Lab で開く
pip install jupyter torch torchvision scikit-learn matplotlib
jupyter notebook inspection_ai_benchmark.ipynb

# または .py に変換して実行
jupyter nbconvert --to script inspection_ai_benchmark.ipynb
python inspection_ai_benchmark.py
```

### デバイス自動対応
| 環境 | デバイス | 備考 |
|------|----------|------|
| Google Colab | CUDA (T4) | 最速。無料枠で利用可 |
| Windows + NVIDIA GPU | CUDA | CUDAドライバ要インストール |
| Windows (GPUなし) | CPU | 動作するが学習に時間がかかる |
| MacBook (M5) | MPS | Apple Silicon加速 |

### 推奨パラメータ調整
| 目的 | 変更するパラメータ |
|------|-------------------|
| 精度向上 | `n_epochs` ↑, `n_train` ↑ |
| SAEスパース化強化 | `sae_l1_weight` ↑ (5e-3 ~ 1e-2) |
| DAE精度重視 | `dae_ssim_weight` ↑, `dae_latent_dim` ↑ |
| 高速検証 | `image_size=64`, `n_epochs=10` |

## 10. 🖼️ テスト画像の読み込みと判定

学習済みモデルに自分の画像を入力して判定結果を確認します。

### セクション10の解説: テスト画像の推論フロー

#### 推論の流れ
```
画像ファイル → PIL画像 → リサイズ(128×128) → グレースケール変換 → テンソル化
    → SAEに入力 → 再構成画像 → 再構成誤差（異常スコア）計算
    → DAEに入力 → 再構成画像 → 再構成誤差（異常スコア）計算
    → 閾値と比較 → 正常/異常の判定
```

#### 3つの実行方法
| 方法 | 用途 | 環境 |
|------|------|------|
| **方法A: 合成テスト** | 動作確認・デモ | どこでも |
| **方法B: アップロード** | 少数画像の手軽なテスト | Google Colab |
| **方法C: フォルダ指定** | 大量画像の一括検査 | Windows / Mac / Colab |

#### 閾値の設定
学習データ（正常品）のスコア分布から「平均 + 3×標準偏差」を自動計算しています。
この閾値を超えるスコアを持つ画像は「異常」と判定されます。
GUIのスライダーで閾値を調整することで、検出感度と誤検知率のバランスを変えられます。

In [ ]:
# ══════════════════════════════════════════════
# テスト画像の読み込み & 推論
# ══════════════════════════════════════════════

import torchvision.transforms as T_transforms
from PIL import Image as PILImage
import glob
from IPython.display import display, HTML

def preprocess_image(pil_img, target_size=128):
    """画像をモデル入力に変換"""
    transform = T_transforms.Compose([
        T_transforms.Resize((target_size, target_size)),
        T_transforms.Grayscale(num_output_channels=1),
        T_transforms.ToTensor(),
    ])
    return transform(pil_img).unsqueeze(0)


def run_inspection(image_paths_or_pils, sae_model, dae_model, device,
                   image_size=128, sae_threshold=None, dae_threshold=None):
    """テスト画像のバッチ推論"""
    sae_model.to(device).eval()
    dae_model.to(device).eval()
    results = []

    for item in image_paths_or_pils:
        if isinstance(item, str):
            name = os.path.basename(item)
            pil = PILImage.open(item).convert('RGB')
        else:
            name = getattr(item, 'filename', 'uploaded')
            pil = item if isinstance(item, PILImage.Image) else PILImage.open(item).convert('RGB')

        # 元画像RGB（リサイズ済み）
        original_rgb = np.array(pil.resize((image_size, image_size))).astype(np.float32) / 255.0

        tensor = preprocess_image(pil, image_size).to(device)
        with torch.no_grad():
            sae_r, _ = sae_model(tensor)
            dae_r, _ = dae_model(tensor)

        sae_score = ((tensor - sae_r)**2).mean().item()
        dae_score = ((tensor - dae_r)**2).mean().item()

        results.append({
            'name': name,
            'pil': pil,
            'original_rgb': original_rgb,
            'input': tensor.squeeze().cpu().numpy(),
            'sae_recon': sae_r.squeeze().cpu().numpy(),
            'dae_recon': dae_r.squeeze().cpu().numpy(),
            'sae_error': torch.abs(tensor - sae_r).squeeze().cpu().numpy(),
            'dae_error': torch.abs(tensor - dae_r).squeeze().cpu().numpy(),
            'sae_score': sae_score,
            'dae_score': dae_score,
            'sae_judge': '異常' if (sae_threshold and sae_score > sae_threshold) else '正常',
            'dae_judge': '異常' if (dae_threshold and dae_score > dae_threshold) else '正常',
        })

    return results


def display_results(results):
    """結果を表とヒートマップで表示（7列: 元画像+エラー+オーバーレイ）"""
    # サマリーテーブル
    print('\n' + '═'*75)
    print('  🖼️ テスト画像 判定結果')
    print('═'*75)
    print(f'{"ファイル":<25} {"SAEスコア":>12} {"SAE判定":>8} {"DAEスコア":>12} {"DAE判定":>8}')
    print('─'*75)
    for r in results:
        sae_icon = '❌' if r['sae_judge']=='異常' else '✅'
        dae_icon = '❌' if r['dae_judge']=='異常' else '✅'
        print(f'{r["name"]:<25} {r["sae_score"]:>12.6f} {sae_icon} {r["sae_judge"]:>4}'
              f' {r["dae_score"]:>12.6f} {dae_icon} {r["dae_judge"]:>4}')
    print('═'*75)

    # 不一致チェック
    disagree = [r for r in results if r['sae_judge'] != r['dae_judge']]
    if disagree:
        print(f'\n⚠️ {len(disagree)}枚で判定不一致:')
        for r in disagree:
            print(f'  📌 {r["name"]}: SAE={r["sae_judge"]} / DAE={r["dae_judge"]}')

    # ヒートマップ（2行×4列）
    n = len(results)
    fig, axes = plt.subplots(n * 2, 4, figsize=(18, n * 6))
    if n == 1:
        axes = axes.reshape(2, 4)

    for i, r in enumerate(results):
        row_top = i * 2
        row_bot = i * 2 + 1

        # 上段: 元画像, 入力(グレー), SAE再構成, DAE再構成
        panels_top = [
            (r['original_rgb'], '元画像', None),
            (r['input'], '入力(グレー)', 'gray'),
            (r['sae_recon'], 'SAE再構成', 'gray'),
            (r['dae_recon'], 'DAE再構成', 'gray'),
        ]
        for j, (p, t, cm) in enumerate(panels_top):
            ax = axes[row_top, j] if n > 1 else axes[0, j]
            if cm: ax.imshow(p, cmap=cm, vmin=0, vmax=1)
            else:  ax.imshow(p)
            if i == 0: ax.set_title(t, fontsize=10, fontweight='bold')
            ax.axis('off')
            if j == 0:
                icon = '🔴' if '異常' in r['sae_judge'] or '異常' in r['dae_judge'] else '🟢'
                ax.set_ylabel(f'{icon}{r["name"][:12]}', fontsize=8, rotation=0, labelpad=70, va='center')

        # 下段: SAEエラー, DAEエラー, SAEオーバーレイ, DAEオーバーレイ
        sae_ov = create_overlay(r['input'], r['sae_error'], alpha=0.55)
        dae_ov = create_overlay(r['input'], r['dae_error'], alpha=0.55)
        panels_bot = [
            (r['sae_error'], 'SAEエラー', True),
            (r['dae_error'], 'DAEエラー', True),
            (sae_ov, 'SAEオーバーレイ', False),
            (dae_ov, 'DAEオーバーレイ', False),
        ]
        bot_titles = ['SAEエラー', 'DAEエラー', 'SAEオーバーレイ', 'DAEオーバーレイ']
        for j, (p, t, use_cmap) in enumerate(panels_bot):
            ax = axes[row_bot, j] if n > 1 else axes[1, j]
            if use_cmap:
                vmax = max(p.max(), 0.001)
                im = ax.imshow(p, cmap=INSPECTION_CMAP, vmin=0, vmax=vmax)
                cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
                cbar.ax.tick_params(labelsize=6)
            else:
                ax.imshow(p)
            if i == 0: ax.set_title(t, fontsize=10, fontweight='bold')
            ax.axis('off')

    plt.suptitle('テスト画像 異常検知結果', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(SAVE_DIR / 'test_image_results.png', dpi=150, bbox_inches='tight')
    plt.show()
plt.close()


print('✅ テスト画像推論関数 定義完了')


In [ ]:
# ══════════════════════════════════════════════
# 方法A: 合成テスト画像で動作確認
# ══════════════════════════════════════════════

# 3σ閾値を計算
sae_threshold = sae_m.get('normal_mean', 0) + 3 * sae_m.get('normal_std', 0.001)
dae_threshold = dae_m.get('normal_mean', 0) + 3 * dae_m.get('normal_std', 0.001)
print(f'閾値: SAE={sae_threshold:.6f}, DAE={dae_threshold:.6f}')

# 合成テスト画像を生成
test_pils = []
for i in range(3):
    arr = generate_normal_image(CONFIG['image_size'], seed=8000+i)
    pil = PILImage.fromarray((arr*255).astype(np.uint8), mode='L').convert('RGB')
    pil.filename = f'normal_{i+1}.png'
    test_pils.append(pil)

for i, dt in enumerate(['scratch', 'stain', 'missing', 'discolor']):
    base = generate_normal_image(CONFIG['image_size'], seed=9000+i)
    defect, _ = add_defect(base, dt, seed=9100+i)
    pil = PILImage.fromarray((defect*255).astype(np.uint8), mode='L').convert('RGB')
    pil.filename = f'defect_{dt}.png'
    test_pils.append(pil)

# 推論実行
results = run_inspection(
    test_pils, sae, dae, DEVICE,
    image_size=CONFIG['image_size'],
    sae_threshold=sae_threshold,
    dae_threshold=dae_threshold,
)
display_results(results)

In [ ]:
# ══════════════════════════════════════════════
# 方法B: Colabにファイルをアップロードして判定
# ══════════════════════════════════════════════

# --- Colabの場合: ファイルアップロードダイアログ ---
try:
    from google.colab import files
    print('📁 画像ファイルを選択してください（複数選択可）')
    print('   対応形式: PNG, JPG, BMP, TIFF')
    uploaded = files.upload()

    if uploaded:
        upload_pils = []
        for fname, content in uploaded.items():
            pil = PILImage.open(io.BytesIO(content)).convert('RGB')
            pil.filename = fname
            upload_pils.append(pil)
            print(f'  ✅ {fname} ({pil.size[0]}x{pil.size[1]})')

        results_upload = run_inspection(
            upload_pils, sae, dae, DEVICE,
            image_size=CONFIG['image_size'],
            sae_threshold=sae_threshold,
            dae_threshold=dae_threshold,
        )
        display_results(results_upload)

except ImportError:
    print('💡 Colab以外の環境です。方法Cでフォルダ指定してください。')
    print('   または上の合成画像テスト（方法A）をお使いください。')

In [ ]:
# ══════════════════════════════════════════════
# 方法C: フォルダ指定で一括判定
# Windows / Mac / Colab(Drive) 共通
# ══════════════════════════════════════════════

# --- パスを変更して使用 ---
# TEST_IMAGE_DIR = r'C:\Users\your_name\test_images'        # Windows
# TEST_IMAGE_DIR = '/Users/your_name/test_images'            # Mac
# TEST_IMAGE_DIR = '/content/drive/MyDrive/test_images'      # Colab + Drive

TEST_IMAGE_DIR = ''  # ← ここにパスを入力

if TEST_IMAGE_DIR and os.path.isdir(TEST_IMAGE_DIR):
    extensions = ['*.png', '*.jpg', '*.jpeg', '*.bmp', '*.tif', '*.tiff']
    image_paths = []
    for ext in extensions:
        image_paths.extend(glob.glob(os.path.join(TEST_IMAGE_DIR, ext)))
        image_paths.extend(glob.glob(os.path.join(TEST_IMAGE_DIR, ext.upper())))
    image_paths = sorted(set(image_paths))

    if image_paths:
        print(f'📂 {len(image_paths)}枚の画像を検出: {TEST_IMAGE_DIR}')
        for p in image_paths[:5]:
            print(f'  📄 {os.path.basename(p)}')
        if len(image_paths) > 5:
            print(f'  ... 他{len(image_paths)-5}枚')

        results_folder = run_inspection(
            image_paths, sae, dae, DEVICE,
            image_size=CONFIG['image_size'],
            sae_threshold=sae_threshold,
            dae_threshold=dae_threshold,
        )
        display_results(results_folder)
    else:
        print(f'⚠️ 画像ファイルが見つかりません: {TEST_IMAGE_DIR}')
else:
    print('💡 TEST_IMAGE_DIR にフォルダパスを指定して実行してください')
    print('   例:')
    print('   Windows:  r"C:\\Users\\user\\test_images"')
    print('   Mac:      "/Users/user/test_images"')
    print('   Colab:    "/content/drive/MyDrive/test_images"')

## 11. 🎛️ インタラクティブGUI（Colab対応）

ボタン操作だけで **画像アップロード → 推論 → 結果レビュー** を行えるGUIです。

> **操作方法**: 下のセルを実行するとGUIが表示されます。

### セクション11の解説: GUIの構成

#### ipywidgetsとは？
Jupyter Notebook / Google Colab上で、ボタン・スライダー・ファイルアップロードなどの
インタラクティブなUI部品を表示するためのライブラリです。
HTMLとCSSを使ったスタイル設定もでき、Pythonだけで簡易的なWebアプリのような体験を実現できます。

#### GUIの操作手順
1. **画像を準備**: ファイルアップロードまたは合成テスト画像を生成
2. **閾値を調整**: スライダーでSAE/DAEそれぞれの判定閾値を設定
3. **推論を実行**: ボタンを押すと全画像を一括判定
4. **結果をレビュー**: サマリーテーブルで全体を確認し、個別画像の詳細ヒートマップを閲覧
5. **CSVエクスポート**: 判定結果をCSVファイルとして保存

#### フィルター機能
- **すべて**: 全画像を表示
- **異常のみ**: SAEまたはDAEで異常判定された画像のみ
- **正常のみ**: 両モデルで正常判定された画像のみ
- **判定不一致のみ**: SAEとDAEで判定が分かれた画像のみ（要注目のケース）

In [ ]:
# ══════════════════════════════════════════════
# GUI コンポーネント（ipywidgets）
# ══════════════════════════════════════════════

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import io as _io
import base64
from PIL import Image as PILImage
import torchvision.transforms as T_transforms

# ── 状態管理 ──
gui_state = {
    'images': [],       # [(name, PIL.Image), ...]
    'results': [],      # run_inspection の出力
    'current_idx': 0,   # 現在表示中の画像index
}

# ── スタイル定義 ──
CSS = HTML('''
<style>
.gui-header { background: linear-gradient(135deg, #1e3a5f, #0f2027);
  color: white; padding: 16px 24px; border-radius: 12px;
  font-size: 18px; font-weight: bold; margin-bottom: 12px; }
.gui-card { background: #f8f9fa; border: 1px solid #e0e0e0;
  border-radius: 8px; padding: 12px; margin: 6px 0; }
.gui-ok { color: #10b981; font-weight: bold; font-size: 16px; }
.gui-ng { color: #ef4444; font-weight: bold; font-size: 16px; }
.gui-warn { color: #f59e0b; font-weight: bold; }
.gui-stat { display: inline-block; background: #e2e8f0;
  border-radius: 6px; padding: 6px 14px; margin: 4px;
  font-family: monospace; font-size: 13px; }
</style>
''')
display(CSS)

print('✅ GUI モジュール読み込み完了')

In [ ]:
# ══════════════════════════════════════════════
# GUI メイン
# ══════════════════════════════════════════════

# ── 出力エリア ──
out_upload_status = widgets.Output()
out_summary = widgets.Output()
out_detail = widgets.Output()
out_heatmap = widgets.Output()

# ── 閾値スライダー ──
sae_thr_default = sae_m.get('normal_mean', 0) + 3 * sae_m.get('normal_std', 0.001)
dae_thr_default = dae_m.get('normal_mean', 0) + 3 * dae_m.get('normal_std', 0.001)

sae_thr_slider = widgets.FloatSlider(
    value=sae_thr_default, min=0.0, max=max(sae_thr_default*5, 0.05),
    step=0.0001, description='SAE 閾値:', readout_format='.5f',
    style={'description_width': '80px'}, layout=widgets.Layout(width='450px'),
)
dae_thr_slider = widgets.FloatSlider(
    value=dae_thr_default, min=0.0, max=max(dae_thr_default*5, 0.05),
    step=0.0001, description='DAE 閾値:', readout_format='.5f',
    style={'description_width': '80px'}, layout=widgets.Layout(width='450px'),
)

# ── ファイルアップローダー ──
file_uploader = widgets.FileUpload(
    accept='.png,.jpg,.jpeg,.bmp,.tif,.tiff',
    multiple=True,
    description='📁 画像を選択',
    layout=widgets.Layout(width='300px'),
)

# ── ボタン ──
btn_run = widgets.Button(
    description='▶ 推論実行', icon='play',
    button_style='primary',
    layout=widgets.Layout(width='160px', height='40px'),
)
btn_synthetic = widgets.Button(
    description='🎲 合成テスト', icon='random',
    button_style='info',
    layout=widgets.Layout(width='160px', height='40px'),
)
btn_prev = widgets.Button(
    description='◀ 前へ', layout=widgets.Layout(width='100px'),
)
btn_next = widgets.Button(
    description='次へ ▶', layout=widgets.Layout(width='100px'),
)
btn_export = widgets.Button(
    description='💾 CSV保存', icon='download',
    button_style='success',
    layout=widgets.Layout(width='160px', height='40px'),
)
label_nav = widgets.HTML(value='<b>─</b>')

# ── フィルター ──
filter_dropdown = widgets.Dropdown(
    options=['すべて', '異常のみ', '正常のみ', '判定不一致のみ'],
    value='すべて', description='表示:',
    style={'description_width': '50px'},
    layout=widgets.Layout(width='220px'),
)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# コールバック関数
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def load_uploaded_files(change=None):
    """アップロードされたファイルを読み込み（ipywidgets新旧両対応）"""
    gui_state['images'] = []
    with out_upload_status:
        clear_output()
        if not file_uploader.value:
            print('📁 画像ファイルを選択してください')
            return

        uploaded = file_uploader.value

        # ipywidgets >= 8: tuple of FileUploadContent objects
        # ipywidgets <  8: dict {filename: {content: bytes, ...}}
        if isinstance(uploaded, dict):
            items = [(fname, meta['content']) for fname, meta in uploaded.items()]
        elif isinstance(uploaded, (list, tuple)):
            items = []
            for item in uploaded:
                if hasattr(item, 'name') and hasattr(item, 'content'):
                    items.append((item.name, item.content))
                elif isinstance(item, dict):
                    items.append((item.get('name', 'unknown'), item.get('content', b'')))
                else:
                    continue
        else:
            print(f'⚠️ 未対応のアップロード形式: {type(uploaded)}')
            return

        for name, content in items:
            try:
                pil = PILImage.open(_io.BytesIO(content)).convert('RGB')
                pil.filename = name
                gui_state['images'].append((name, pil))
            except Exception as e:
                print(f'  ⚠️ {name}: 読み込みエラー ({e})')
        print(f'✅ {len(gui_state["images"])}枚の画像を読み込みました')
        for n, p in gui_state['images']:
            print(f'  📄 {n} ({p.size[0]}×{p.size[1]})')

file_uploader.observe(load_uploaded_files, names='value')


def on_synthetic_click(btn):
    """合成テスト画像を生成"""
    gui_state['images'] = []
    with out_upload_status:
        clear_output()
        print('🎲 合成テスト画像を生成中...')
        for i in range(3):
            arr = generate_normal_image(CONFIG['image_size'], seed=8000+i)
            pil = PILImage.fromarray((arr*255).astype(np.uint8), mode='L').convert('RGB')
            pil.filename = f'normal_{i+1}.png'
            gui_state['images'].append((f'normal_{i+1}.png', pil))
        for i, dt in enumerate(['scratch', 'stain', 'missing', 'discolor']):
            base = generate_normal_image(CONFIG['image_size'], seed=9000+i)
            defect, _ = add_defect(base, dt, seed=9100+i)
            pil = PILImage.fromarray((defect*255).astype(np.uint8), mode='L').convert('RGB')
            pil.filename = f'defect_{dt}.png'
            gui_state['images'].append((f'defect_{dt}.png', pil))
        print(f'✅ {len(gui_state["images"])}枚生成（正常3枚 + 異常4枚）')

btn_synthetic.on_click(on_synthetic_click)


def on_run_click(btn):
    """推論実行"""
    if not gui_state['images']:
        with out_summary:
            clear_output()
            print('⚠️ 画像がありません。アップロードまたは合成テストを実行してください。')
        return

    # モデルをDEVICEに同期（CPU/CUDA不一致防止）
    sae.to(DEVICE).eval()
    dae.to(DEVICE).eval()

    pils = [p for _, p in gui_state['images']]
    gui_state['results'] = run_inspection(
        pils, sae, dae, DEVICE,
        image_size=CONFIG['image_size'],
        sae_threshold=sae_thr_slider.value,
        dae_threshold=dae_thr_slider.value,
    )
    gui_state['current_idx'] = 0
    update_summary()
    update_detail_view()

btn_run.on_click(on_run_click)


def get_filtered_results():
    """フィルター適用"""
    results = gui_state['results']
    f = filter_dropdown.value
    if f == '異常のみ':
        return [(i,r) for i,r in enumerate(results) if '異常' in r['sae_judge'] or '異常' in r['dae_judge']]
    elif f == '正常のみ':
        return [(i,r) for i,r in enumerate(results) if r['sae_judge']=='正常' and r['dae_judge']=='正常']
    elif f == '判定不一致のみ':
        return [(i,r) for i,r in enumerate(results) if r['sae_judge'] != r['dae_judge']]
    return list(enumerate(results))


def update_summary():
    """サマリーダッシュボード更新"""
    with out_summary:
        clear_output(wait=True)
        results = gui_state['results']
        if not results:
            return
        n = len(results)
        n_sae_ng = sum(1 for r in results if '異常' in r['sae_judge'])
        n_dae_ng = sum(1 for r in results if '異常' in r['dae_judge'])
        n_disagree = sum(1 for r in results if r['sae_judge'] != r['dae_judge'])

        html = '<div class="gui-card">'
        html += '<b>📊 判定サマリー</b><br><br>'
        html += f'<span class="gui-stat">検査数: <b>{n}</b>枚</span>'
        html += f'<span class="gui-stat">SAE異常: <b class="gui-ng">{n_sae_ng}</b>/{n}</span>'
        html += f'<span class="gui-stat">DAE異常: <b class="gui-ng">{n_dae_ng}</b>/{n}</span>'
        if n_disagree:
            html += f'<br><br><span class="gui-warn">⚠️ {n_disagree}枚で判定不一致</span>'
        html += '</div>'

        # テーブル
        html += '<table style="width:100%; border-collapse:collapse; font-size:13px; margin-top:8px;">'
        html += '<tr style="background:#334155; color:white;">'
        html += '<th style="padding:6px 8px; text-align:left;">ファイル</th>'
        html += '<th>SAEスコア</th><th>SAE</th>'
        html += '<th>DAEスコア</th><th>DAE</th><th></th></tr>'
        for i, r in enumerate(results):
            bg = '#fff5f5' if ('異常' in r['sae_judge'] or '異常' in r['dae_judge']) else '#f0fdf4'
            sae_cls = 'gui-ng' if '異常' in r['sae_judge'] else 'gui-ok'
            dae_cls = 'gui-ng' if '異常' in r['dae_judge'] else 'gui-ok'
            warn = ' ⚠️' if r['sae_judge'] != r['dae_judge'] else ''
            html += f'<tr style="background:{bg}; border-bottom:1px solid #e2e8f0;">'
            html += f'<td style="padding:5px 8px;">{r["name"]}</td>'
            html += f'<td style="text-align:center; font-family:monospace;">{r["sae_score"]:.6f}</td>'
            html += f'<td style="text-align:center;"><span class="{sae_cls}">{r["sae_judge"]}</span></td>'
            html += f'<td style="text-align:center; font-family:monospace;">{r["dae_score"]:.6f}</td>'
            html += f'<td style="text-align:center;"><span class="{dae_cls}">{r["dae_judge"]}</span>{warn}</td>'
            html += f'<td style="text-align:center;">#{i+1}</td></tr>'
        html += '</table>'
        display(HTML(html))


def update_detail_view():
    """個別画像の詳細ビュー更新"""
    filtered = get_filtered_results()
    if not filtered:
        with out_detail:
            clear_output()
            print('該当する画像がありません')
        return

    idx = gui_state['current_idx'] % len(filtered)
    orig_i, r = filtered[idx]
    label_nav.value = f'<b>{idx+1} / {len(filtered)}</b>'

    with out_detail:
        clear_output(wait=True)
        sae_cls = 'gui-ng' if '異常' in r['sae_judge'] else 'gui-ok'
        dae_cls = 'gui-ng' if '異常' in r['dae_judge'] else 'gui-ok'
        html = f'<div class="gui-card">'
        html += f'<b>📄 {r["name"]}</b> (#{orig_i+1})<br><br>'
        html += f'<span class="gui-stat">SAE: <span class="{sae_cls}">{r["sae_judge"]}</span> ({r["sae_score"]:.6f})</span> '
        html += f'<span class="gui-stat">DAE: <span class="{dae_cls}">{r["dae_judge"]}</span> ({r["dae_score"]:.6f})</span>'
        if r['sae_judge'] != r['dae_judge']:
            html += '<br><span class="gui-warn">⚠️ モデル間で判定不一致</span>'
        html += '</div>'
        display(HTML(html))

    with out_heatmap:
        clear_output(wait=True)
        # Replaced by 2x4 layout below
        # 上段: 元画像, 入力, SAE再構成, DAE再構成
        fig, axes = plt.subplots(2, 4, figsize=(18, 7))
        top_panels = [
            (r.get('original_rgb', r['input']), '元画像', None),
            (r['input'], '入力(グレー)', 'gray'),
            (r['sae_recon'], 'SAE再構成', 'gray'),
            (r['dae_recon'], 'DAE再構成', 'gray'),
        ]
        for j, (p, t, cm) in enumerate(top_panels):
            ax = axes[0, j]
            if cm: ax.imshow(p, cmap=cm, vmin=0, vmax=1)
            else:  ax.imshow(p)
            ax.set_title(t, fontsize=10, fontweight='bold')
            ax.axis('off')

        # 下段: SAEエラー, DAEエラー, SAEオーバーレイ, DAEオーバーレイ
        sae_ov = create_overlay(r['input'], r['sae_error'], alpha=0.55)
        dae_ov = create_overlay(r['input'], r['dae_error'], alpha=0.55)
        bot_panels = [
            (r['sae_error'], 'SAEエラー', True),
            (r['dae_error'], 'DAEエラー', True),
            (sae_ov, 'SAEオーバーレイ', False),
            (dae_ov, 'DAEオーバーレイ', False),
        ]
        for j, (p, t, use_cmap) in enumerate(bot_panels):
            ax = axes[1, j]
            if use_cmap:
                vmax = max(p.max(), 0.001)
                im = ax.imshow(p, cmap=INSPECTION_CMAP, vmin=0, vmax=vmax)
                cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
                cbar.ax.tick_params(labelsize=6)
            else:
                ax.imshow(p)
            ax.set_title(t, fontsize=10, fontweight='bold')
            ax.axis('off')

        plt.tight_layout()
        plt.show()
        plt.close()


def on_prev(btn):
    filtered = get_filtered_results()
    if filtered:
        gui_state['current_idx'] = (gui_state['current_idx'] - 1) % len(filtered)
        update_detail_view()

def on_next(btn):
    filtered = get_filtered_results()
    if filtered:
        gui_state['current_idx'] = (gui_state['current_idx'] + 1) % len(filtered)
        update_detail_view()

btn_prev.on_click(on_prev)
btn_next.on_click(on_next)


def on_filter_change(change):
    gui_state['current_idx'] = 0
    update_detail_view()

filter_dropdown.observe(on_filter_change, names='value')


def on_export_csv(btn):
    """結果をCSVで保存"""
    if not gui_state['results']:
        return
    import csv
    csv_path = SAVE_DIR / 'test_results.csv'
    with open(csv_path, 'w', newline='', encoding='utf-8-sig') as f:
        w = csv.writer(f)
        w.writerow(['ファイル名','SAEスコア','SAE判定','DAEスコア','DAE判定','不一致'])
        for r in gui_state['results']:
            w.writerow([
                r['name'], f'{r["sae_score"]:.6f}', r['sae_judge'],
                f'{r["dae_score"]:.6f}', r['dae_judge'],
                '⚠' if r['sae_judge']!=r['dae_judge'] else '',
            ])
    with out_upload_status:
        print(f'\n💾 CSV保存完了: {csv_path}')
    # Colab自動ダウンロード
    try:
        from google.colab import files as colab_files
        colab_files.download(str(csv_path))
    except ImportError:
        pass

btn_export.on_click(on_export_csv)


# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# GUIレイアウト組み立て
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
header = widgets.HTML('<div class="gui-header">🔬 外観検査AI — テスト画像レビュー</div>')

upload_section = widgets.VBox([
    widgets.HTML('<b>① 画像を準備</b>'),
    widgets.HBox([file_uploader, btn_synthetic]),
    out_upload_status,
])

threshold_section = widgets.VBox([
    widgets.HTML('<b>② 閾値設定</b>'),
    sae_thr_slider,
    dae_thr_slider,
    widgets.HBox([btn_run, btn_export]),
])

nav_section = widgets.HBox(
    [btn_prev, label_nav, btn_next, filter_dropdown],
    layout=widgets.Layout(align_items='center', gap='12px'),
)

review_section = widgets.VBox([
    widgets.HTML('<b>③ 結果レビュー</b>'),
    out_summary,
    widgets.HTML('<hr>'),
    widgets.HTML('<b>④ 個別画像詳細</b>'),
    nav_section,
    out_detail,
    out_heatmap,
])

gui = widgets.VBox([
    header,
    upload_section,
    widgets.HTML('<hr>'),
    threshold_section,
    widgets.HTML('<hr>'),
    review_section,
], layout=widgets.Layout(max_width='900px'))

display(gui)
print('\n💡 操作手順: ①画像を選択 → ②閾値を調整 → ▶推論実行 → ③結果を確認 → ◀▶で画像を切替')